## Load and prepare the data

In [1]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd = o3d.io.read_point_cloud("pc_assignment\shoe_pc.ply")

In [3]:
# Estimate normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# Optionally, orient the normals consistently
pcd.orient_normals_consistent_tangent_plane(100)

In [4]:
# Save the point cloud with normals
#o3d.io.write_point_cloud("output\output_with_normals_shoe_pc.ply", pcd)

# Access points, colors, and normals if needed
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
normals = np.asarray(pcd.normals)

# Algorithm 1: Ball Pivoting Algorithm

In [6]:
#radius determination
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [7]:
#computing the mehs
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

In [8]:
#export
# o3d.io.write_triangle_mesh("output\pc_assignment\\bpa_mesh.ply", bpa_mesh)
o3d.visualization.draw_geometries([bpa_mesh])

In [9]:
#decimating the mesh
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

In [10]:
#export
# o3d.io.write_triangle_mesh("output\pc_assignment\\dec_mesh.ply", dec_mesh)
o3d.visualization.draw_geometries([dec_mesh])

*Optional ---*

In [11]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

TriangleMesh with 184803 points and 54649 triangles.

In [12]:
#export
# o3d.io.write_triangle_mesh("output\pc_assignment\\opt_dec_mesh.ply", dec_mesh)
o3d.visualization.draw_geometries([dec_mesh])

# Algorithm 2: Poisson' reconstruction

In [6]:
#computing the mesh
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [7]:
#export
# o3d.io.write_triangle_mesh("output\pc_assignment\p_mesh.ply", poisson_mesh)
o3d.visualization.draw_geometries([poisson_mesh])

In [15]:
#cropping
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

In [16]:
#export and visualize
# o3d.io.write_triangle_mesh("output\pc_assignment\\p_mesh_crop.ply", p_mesh_crop)
o3d.visualization.draw_geometries([p_mesh_crop])